In [ ]:
import os

import gpr

import numpy as np
import scipy.optimize

In [ ]:
theta0 = np.array([200, 0.1, 0.1, np.pi/2, 1])
bounds = np.array([(0.001, 1000), (0.0001, 10), (0.0001, 10), (0, np.pi), (0.1, 10)])

def g1(x):
    sigma_x = x[1]
    sigma_y = x[2]
    return -sigma_x + 50 * sigma_y + 1

def g2(x):
    sigma_x = x[1]
    sigma_y = x[2]
    return -sigma_y + 50 * sigma_x + 1

In [ ]:
nLML_print = True
nLML_factor = 1e3
datafile = 'folio2'
# nExposure = 450
nSigma = 4
sample = None
train_size = 0.50
epsilon = 0.0001

exposures = np.arange(458, 500)
for nExposure in exposures:
    uGP = gpr.GPR('dx', nLML_print=nLML_print, nLML_factor=nLML_factor)
    uGP.extract(datafile, nExposure, nSigma, sample=sample)
    uGP.split_data(train_size)

    vGP = gpr.GPR('dy', nLML_print=nLML_print, nLML_factor=nLML_factor)
    vGP.extract(datafile, nExposure, nSigma, sample=sample)
    vGP.split_data(train_size)

    uresult = scipy.optimize.fmin_slsqp(
        uGP.get_nLML,
        theta0,
        ieqcons=[g1, g2],
        bounds=bounds,
#         iter=30,
        iprint=2,
        epsilon=epsilon
    )    
    uGP.fit(uresult)

    vresult = scipy.optimize.fmin_slsqp(
        vGP.get_nLML,
        uresult,
        ieqcons=[g1, g2],
        bounds=bounds,
#         iter=30,
        iprint=2,
        epsilon=epsilon
    )    
    vGP.fit(vresult)

    GP = uGP.combine(vGP)
    GP.error()

    np.savez(
        os.path.join("../exposures", f"{nExposure}.npz"),
        fbar_s=GP.fbar_s,
        sigma=GP.sigma,
        nLML=GP.nLML,
        utheta=np.array(GP.utheta),
        vtheta=np.array(GP.vtheta),
        random_state=GP.random_state,
        nExposure=GP.nExposure,
        nTrain=GP.nTrain,
        nTest=GP.nTest,
        nData=GP.nData,
        nParams=GP.nParams,
        X=GP.X,
        Xtrain=GP.Xtrain,
        Xtest=GP.Xtest,
        Y=GP.Y,
        Ytrain=GP.Ytrain,
        Ytest=GP.Ytest,
        E=GP.E,
        Etrain=GP.Etrain,
        Etest=GP.Etest,
        t0=GP.t0,
        tf=GP.tf,
        RSS=GP.RSS,
        chisq=GP.chisq,
        red_chisq=GP.red_chisq
    )